Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.
Мы обучим модель классифицировать комментарии на позитивные и негативные. 

В нашем распоряжении набор данных с разметкой о токсичности правок. Мы будем использовать следующие модели:
- дерево решений
- случайный лес
- логистическая регрессия
- стохастический градиентный спуск

## Подготовка данных



In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score
from nltk.stem import WordNetLemmatizer 
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
nltk.download('stopwords')
from nltk.corpus import stopwords
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
display(data.head())

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data = data.drop('Unnamed: 0', axis=1)

In [6]:
data.groupby(by='toxic')['toxic'].count()

toxic
0    143106
1     16186
Name: toxic, dtype: int64

Мы видим, что в датасете содержится около 16000 комментариев, из которых примерно 10% - токсичные. Мы лемматизируем комментарии с помощью WordNetLemmatizer, дополнительно используя POC-теги (мы использовали метод из статьи https://webdevblog.ru/podhody-lemmatizacii-s-primerami-v-python/).

In [7]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [8]:
def lemmatize_text(sentence):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(sentence)
    return ' '.join([lemmatizer.lemmatize(w.lower(), get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])

In [9]:
tqdm.pandas()

In [10]:
data['lemmatized_text'] = data['text'].progress_apply(lemmatize_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

Разобьем данные выборки на тренировочную, валидационную и тестовую часть в соотношении 8:1:1.

In [11]:
features = data['lemmatized_text']
target = data['toxic']

In [12]:
comments_train, comments_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.2, random_state=12345, stratify=target)


In [13]:
comments_valid, comments_test, target_valid, target_test = train_test_split(
    comments_valid_test, target_valid_test, test_size=0.5, random_state=12345, stratify=target_valid_test)


In [17]:
print(comments_train.shape, target_train.shape, comments_valid.shape, target_valid.shape, comments_test.shape, target_test.shape)

(127433,) (127433,) (15929,) (15929,) (15930,) (15930,)


Создадим TF-IDF матрицу, которую будем использовать в качестве матрицы признаков.

In [14]:
stoplist = set(stopwords.words('english'))


In [15]:
count_tf_idf = TfidfVectorizer(stop_words=stoplist) 
tf_idf = count_tf_idf.fit_transform(comments_train) 
features_train = tf_idf
features_valid = count_tf_idf.transform(comments_valid)
features_test = count_tf_idf.transform(comments_test)

## Обучение моделей

### Дерево решений
Мы видим, что при большой глубине дерева решений значение f1_score достаточно велико (0.68), но всё равно не дотягивает до целевого значения в 0.75, поэтому считаем нерациональным дальнейшие эксперименты с глубиной дерева.

In [22]:
dtc = DecisionTreeClassifier(random_state = 12345)
grid = {'max_depth': (30,40,10)}
gdtc = GridSearchCV(dtc, param_grid=grid, scoring='f1', n_jobs=-1)
gdtc.fit(features_train, target_train)
best_model_dtc = gdtc.best_estimator_
predictions_valid = best_model_dtc.predict(features_valid)
f1_tree = f1_score(target_valid, predictions_valid)
accuracy_tree = accuracy_score(target_valid, predictions_valid)


In [23]:
print('На валидационной выборке (метод DecisionTreeClassifier) f1 =', f1_tree)
print('На валидационной выборке (метод DecisionTreeClassifier) accuracy =', accuracy_tree)

На валидационной выборке (метод DecisionTreeClassifier) f1 = 0.6834061135371179
На валидационной выборке (метод DecisionTreeClassifier) accuracy = 0.9453826354447863


### Случайный лес
Модель даёт низкое значение f1_score и не подходит в данной задаче.

In [24]:
rfc = RandomForestClassifier(random_state = 12345)
grid = {'n_estimators': (1,5,1), 'max_depth': (10,20,2)}
grfc = GridSearchCV(rfc, param_grid=grid, scoring='f1', n_jobs=-1)
grfc.fit(features_train, target_train)
best_model_rfc = grfc.best_estimator_
predictions_valid = best_model_rfc.predict(features_valid)
f1_forest = f1_score(target_valid, predictions_valid)
accuracy_forest = accuracy_score(target_valid, predictions_valid)


In [25]:
print('На валидационной выборке (метод RandomForestClassifier) f1 =', f1_forest)
print('На валидационной выборке (метод RandomForestClassifier) accuracy =', accuracy_forest)

На валидационной выборке (метод RandomForestClassifier) f1 = 0.06611570247933883
На валидационной выборке (метод RandomForestClassifier) accuracy = 0.9006842865214388


### Логистическая регрессия
Модель логистической регрессии позволяет достичь целевого значения f1 (f1=0.76), но дополнительно применим метод изменения порога классификации.

In [26]:
lr = LogisticRegression(random_state = 12345, class_weight = 'balanced', solver='liblinear', max_iter = 1000)
grid = {'C':(5,15)}
glr = GridSearchCV(lr, param_grid=grid, scoring='f1', n_jobs=-1)
glr.fit(features_train, target_train)
best_model_lr = glr.best_estimator_
predictions_valid = best_model_lr.predict(features_valid)
f1_regression = f1_score(target_valid, predictions_valid)
accuracy_regression = accuracy_score(target_valid, predictions_valid)

In [27]:
print('На валидационной выборке (метод LogisticRegression) f1 =', f1_regression)
print('На валидационной выборке (метод LogisticRegression) accuracy =', accuracy_regression)

На валидационной выборке (метод LogisticRegression) f1 = 0.7607223476297967
На валидационной выборке (метод LogisticRegression) accuracy = 0.9467637642036537


In [29]:
probabilities_test = best_model_lr.predict_proba(features_test)
probabilities_valid = best_model_lr.predict_proba(features_valid)

best_f1 = 0
best_threshold_lr = 0
for threshold in np.arange(0, 1, 0.01):
    predictions_valid  = probabilities_valid[:,1] > threshold
    if f1_score(target_valid, predictions_valid) > best_f1:
        best_threshold_lr = threshold
        best_f1 = f1_score(target_valid, predictions_valid)
        
predictions_valid  = probabilities_valid[:,1] > best_threshold_lr
   
f1_regression_threshold = f1_score(target_valid, predictions_valid)
accuracy_regression_threshold = accuracy_score(target_valid, predictions_valid)


In [30]:
print('На валидационной выборке (метод LogisticRegression + подбор порога классификации) f1 =', f1_regression_threshold)
print('На валидационной выборке (метод LogisticRegression + подбор порога классификации) accuracy =', accuracy_regression_threshold)

На валидационной выборке (метод LogisticRegression + подбор порога классификации) f1 = 0.7902408111533588
На валидационной выборке (метод LogisticRegression + подбор порога классификации) accuracy = 0.9584405800740787


### Стохастический градиентный спуск
Как и логистическая регрессия, стохастический градиентный спуск дает значение f1 близкое к целевому, а после дополнительного подбора порога классификации f1=0.78.

In [36]:
sgdc = SGDClassifier(random_state = 12345, loss='modified_huber', penalty="l2", class_weight = 'balanced', max_iter = 1000)
grid = {}
gsgdc = GridSearchCV(sgdc, param_grid=grid, scoring='f1', n_jobs=-1)
gsgdc.fit(features_train, target_train)
best_model_sgdc = gsgdc.best_estimator_
predictions_valid = best_model_sgdc.predict(features_valid)
f1_SGDC =  f1_score(target_valid, predictions_valid)
accuracy_SGDC =  accuracy_score(target_valid, predictions_valid)

In [37]:
print('На валидационной выборке (метод SGDClassifier) f1 =', f1_SGDC)
print('На валидационной выборке (метод SGDClassifier) accuracy =', accuracy_SGDC)

На валидационной выборке (метод SGDClassifier) f1 = 0.7463984778472411
На валидационной выборке (метод SGDClassifier) accuracy = 0.9414275849080294


In [38]:
probabilities_valid = best_model_sgdc.predict_proba(features_valid)

best_f1 = 0
best_threshold_SGD = 0
for threshold in np.arange(0, 1, 0.01):
    predictions_valid  = probabilities_valid[:,1] > threshold
    if f1_score(target_valid, predictions_valid) > best_f1:
        best_threshold_SGD = threshold
        best_f1 = f1_score(target_valid, predictions_valid)
        
predictions_valid  = probabilities_valid[:,1] > best_threshold_SGD
   
f1_SGDC_threshold = f1_score(target_valid, predictions_valid)
accuracy_SGDC_threshold = accuracy_score(target_valid, predictions_valid)

In [39]:
print('На валидационной выборке (метод SGDClassifier + подбор порога классификации) f1 =', f1_SGDC_threshold)
print('На валидационной выборке (метод SGDClassifier + подбор порога классификации) accuracy =', accuracy_SGDC_threshold)

На валидационной выборке (метод SGDClassifier + подбор порога классификации) f1 = 0.7932354818123802
На валидационной выборке (метод SGDClassifier + подбор порога классификации) accuracy = 0.959319480193358


### Проверка моделей на адекватность, сравнение с константной моделью
Составим вектор предсказаний, состоящий из нулей (нет токсичных комментариев). Мы видим, что значение accuracy составляет около 0.9 (10% токсичных комментариев, как мы и видели в датасете), при этом accuracy ниже, чем для моделей логистической регрессии и стохастического спуска, таким образом, построенные выше модели адекватны.

In [40]:
predictions_const = [0]*len(target_valid)
accuracy_const = accuracy_score(target_valid, predictions_const)

In [41]:
print(accuracy_const)

0.8984242576432921


## Вывод и итоговое тестирование
Лучшими моделями оказались модели **стохастического градиентного спуска** и **логистической регрессии**, но обе дополнительно потребовали подбора порога классификации. Проведем итоговое тестирование на двух лучших моделях и составим таблицу. Видим, что обе модели проходят итоговое тестирование со значением f1=0.78.

In [42]:
probabilities_test_lr = best_model_lr.predict_proba(features_test)
predictions_test_lr  = probabilities_test_lr[:,1] > best_threshold_lr
f1_lr_test =  f1_score(target_test, predictions_test_lr)

In [44]:
probabilities_test_SGD = best_model_sgdc.predict_proba(features_test)
predictions_test_SGD  = probabilities_test_SGDC[:,1] > best_threshold_SGD
f1_SGDC_test =  f1_score(target_test, predictions_test_SGD)

In [49]:
result = pd.DataFrame(data = [['DecisionTreeClassifier', f1_tree, accuracy_tree, '-'],
                              ['RandomForestClassifier', f1_forest, accuracy_forest, '-'],
                              ['LogisticRegression + threshold', f1_regression_threshold, accuracy_regression_threshold, f1_lr_test],
                              ['SGDClassifier + threshold', f1_SGDC_threshold, accuracy_SGDC_threshold, f1_SGDC_test],
                              ['Constant', '-', accuracy_const, '-']],                     
                      columns = ['Model', 'f1_score_valid', 'accuracy_score_valid', 'f1_score_test'])

In [50]:
display(result)

,Model,f1_score_valid,accuracy_score_valid,f1_score_test
0,DecisionTreeClassifier,0.683406,0.945383,-
1,RandomForestClassifier,0.066116,0.900684,-
2,LogisticRegression + threshold,0.790241,0.958441,0.782636
3,SGDClassifier + threshold,0.793235,0.959319,0.780365
4,Constant,-,0.898424,-
